<a href="https://colab.research.google.com/github/newbee1905/TrafficFlowPrediction/blob/dev%2Fnewbee1905/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!if [ -d "TrafficFlowPrediction" ]; then rm -rf TrafficFlowPrediction; fi
!git config --global user.name "newbee1905"
!git clone https://github.com/newbee1905/TrafficFlowPrediction -b dev/newbee1905

Cloning into 'TrafficFlowPrediction'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 132 (delta 45), reused 76 (delta 30), pack-reused 32
Receiving objects: 100% (132/132), 13.47 MiB | 25.36 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [3]:
%cd TrafficFlowPrediction
!pip install -r requirements.txt

/content/TrafficFlowPrediction
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.6/254.6 kB 30.2 MB/s eta 0:00:00
     ━

In [ ]:
!python train.py --model gru

In [1]:
!python train.py --model lstm

python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [ ]:
!python train.py --model saes

In [ ]:
!python main.py